Labor force effect on import production
=====================================================================================

## Introduction
This notebook aims to provide insight on the practically of the United State of America (USA) being able to produce it's imports products and services internal thereby improving the gross domestic product (GDP) of the USA




##  Objectives


1.   How many productive hours will it take the USA to produce its imports internally.
2.   What is the percentage increase in the hours worked if the imports are produced internally.
3.   What is the implication of this on the labour force in the USA

## Prerequisites
To follow along, you should have a basic understanding of Python, data analysis libraries like Pandas, numpy etc and knowledge of Scikit learn for Marching Learning.


# Data
Here is a list of data used to achieve the objective of this study

1.   Trade Data - Both goods and services - https://comtradeplus.un.org/
2.   GDP at constant 2015 - https://databank.worldbank.org/source/world-development-indicators#
3.   GDP per Capita - https://databank.worldbank.org/source/world-development-indicators#
4.   Population - https://rshiny.ilo.org/dataexplorer22/?lang=en&id=HOW_UEMP_SEX_NB_A
5.   Labor Force Data - https://rshiny.ilo.org/dataexplorer22/?lang=en&id=HOW_UEMP_SEX_NB_A
6.   Export value index (2015 = 100) - https://data.worldbank.org/indicator/TX.VAL.MRCH.XD.WD



## Data Description

## GDP data
This data records the montary and market value of all finished good and services produce within a country at a perticular point in time.

### GDP per capita Data
This data is determine by the amount generate per household within an ecomomy. This is gotten by dividing the GDP by the toal population within an economy. It will be used to train the dataset to determine or predict the GDP per hour for countries the GDPper hour can not be determined for.

### Trade Data
This notebook leverages the United Nations Comtrade Database and OECD database to explore import data for the United States.

### Products data
The United Nations Comtrade Database is one of the world’s largest and most comprehensive sources of international trade data, maintained by the United Nations Statistics Division (UNSD). It provides detailed information on trade flows between countries, covering thousands of products and services traded worldwide. - https://comtradeplus.un.org/

### Services data
Trade in services records the value of services exchanged between residents and non-residents of an economy, including services provided through foreign affiliates established abroad. - https://www.oecd.org/en/data/indicators/trade-in-services.

## Export value index (2015 = 100)
https://databank.worldbank.org/source/world-development-indicators

The Export Value Index (EVI) is an economic indicator that measures changes in the value of a country’s exports over time, with 2015 set as the base year (index = 100). EVI captures both price changes (e.g., inflation or price changes of commodities) and changes in the volume of exported goods, providing a broad measure of export trends.


In [3]:
# Import pacakged
import requests
import pandas as pd
import numpy as np
import json
from io import StringIO
import plotly.express as px
import statsmodels.api as sm
import comtradeapicall

#model trainning packages
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error



# Data extraction
we are going to load the following datasets .....

In [4]:
 #API Calls and Data Calls
working_age_population_url = 'https://rplumber.ilo.org/data/indicator/?id=POP_XWAP_SEX_AGE_NB_A&type=label&format=.csv'
avg_weekly_hours_url = 'https://rplumber.ilo.org/data/indicator/?id=HOW_XEES_SEX_NB_A&type=label&format=.csv'
country_code_db = pd.read_csv("Data/country_code_db.csv")
GDP = pd.read_csv("Data/GDP at 2015 constant price 2022.csv")
GDP_per_capita = pd.read_excel('Data/GDP per capita constant 2015.xlsx', sheet_name='Data')

#Comtrade Data call
f = open('Data/HS (1).json',encoding='utf-8')
path2 = 'Data/country_codes_V202401b (2).csv'

# export index
index_df = pd.read_csv('Data/d7f111f0-1329-4004-a6dc-a355f2385ec2_Data.csv')

# service Data
services_df = pd.read_csv('Data/Commercial Services 7.31.2024.csv')



## Confirm that data extraction for working population and average weekly hours are successfully extracted

In [5]:

# Fetching the data from the ILO API
working_population_response = requests.get(working_age_population_url)
avg_weekly_hours_response   = requests.get(avg_weekly_hours_url)

# Check for successful working_population_response
if working_population_response.status_code == 200:
    try:
        wp_text= working_population_response.text
        data = StringIO(wp_text)
        # Convert dataset to DataFrame
        print("Data fetched successfully!")
        wp_df = pd.read_csv(data)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        print(working_population_response.text)  # Debug by printing raw response
else:
     print(f"Failed to fetch data: {working_population_response.status_code}")


# Check for successful averager_work_hours_response
if avg_weekly_hours_response.status_code == 200:
    try:
        wh_text= avg_weekly_hours_response.text
        data = StringIO(wh_text)
        # Convert dataset to DataFrame
        print("Data fetched successfully!")
        wh_df = pd.read_csv(data)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        print(avg_weekly_hours_response.text)  # Debug by printing raw response
else:
    print(f"Failed to fetch data: {avg_weekly_hours_response.status_code}")


Data fetched successfully!
Data fetched successfully!


## Helper function to clearn extracted data for working population, avg weekly working hours and GDP

In [6]:
# gdp_per_hour calculation
class GDPPerHour:
    """
    This class gdp-per_hour is the algorithm for cleaning, calculating and trainning gdp per hour.
    It will take each data sources. These sources are
    1. International Labour Organisation (ILO) data for working age population
    2. ILO data for average weekly hours worked
    3. CSV download of GDP from the Worldbank
    """
    def __init__(self,wp_data = 'None', wh_data = 'None', GDP = 'None'):
        self.wp_data = wp_data
        self.wh_data = wh_data
        self.GDP = GDP


    """
    Function for Data cleaning and exploration for working population
    """

    #Function to clean data for working population
    def Clean_wp_data(self):

        # Copy the data to avoid changing the original DataFrame outside the function scope
        wp_data = self.wp_data.copy()

        # change columns name
        wp_data.rename(columns={
                        'ref_area.label': 'country',
                        'source.label': 'source',
                        'obs_value' : 'no_working_population',
                        'indicator.label' : 'description',
                        'sex.label' : 'gender',
                        'time':'year',
                        'note_source.label': 'note_to_description',
                        'classif1.label':'category'
                    },inplace=True)

        #drop unused columns
        columns_to_drop = ['obs_status.label', 'note_classif.label','note_indicator.label']
        wp_data.drop(columns=columns_to_drop, inplace=True)

        #Multiply the no_working_population by a thousand
        wp_data['no_working_population'] = wp_data['no_working_population'].apply(lambda x: x*1000)

        # Remove the first 5 characters from the 'code' column
        wp_data['gender'] = wp_data['gender'].str[5:]

        # #filter the gender to only select totals and category data to only select Age (Youth, adults): 15+
        filtered_data = wp_data[
            (wp_data['gender'] == 'Total') &
            (wp_data['category'] == 'Age (Youth, adults): 15+') &
            (wp_data['year'] <= 2022)
        ]
        # Sort the filter_data by country and year
        filtered_data.sort_values(by=['country', 'year'], inplace=True)

        # Create 'RowNumber' within each 'Category' partition
        filtered_data['RowNumber'] = filtered_data.groupby('country').cumcount() + 1

        # Find the maximum RowNumber for each Country
        max_rownumber = filtered_data.groupby('country')['RowNumber'].max().reset_index()

        # Merge to get rows with the max RowNumber per country
        filtered_data = pd.merge(filtered_data, max_rownumber, on=['country', 'RowNumber'], how='inner')

        return filtered_data

    """
    Function for Data cleaning and exploration for avaragle weekly hours converted to yearly weekly hours
    """
    #Function to clean data for average weekly hours and convert it to average annual hours
    def Clean_wh_data(self):

        # Copy the data to avoid changing the original DataFrame outside the function scope
        wh_data = self.wh_data.copy()
        # change columns name
        wh_data.rename(columns={
                        'ref_area.label': 'country',
                        'source.label': 'source',
                        'obs_value' : 'average_weekly_hours',
                        'indicator.label' : 'description',
                        'sex.label' : 'gender',
                        'time':'year',
                        'note_source.label': 'note_to_description',
                    },inplace=True)

        #drop unused columns
        columns_to_drop = ['obs_status.label','note_indicator.label']
        wh_data.drop(columns=columns_to_drop, inplace=True)

        #Create a new column and multiply the no_working_population by a thousand
        wh_data['average_yearly_hours'] = wh_data['average_weekly_hours'].apply(lambda x: x*52)

        # Remove the first 5 characters from the 'code' column
        wh_data['gender'] = wh_data['gender'].str[5:]

        # #filter the gender to only select totals and category data to only select Age (Youth, adults): 15+
        filtered_data = wh_data[
        (wh_data['gender'] == 'Total') &
        (wh_data['year'] <= 2022)
        ]
        # Sort the filter_data by country and year
        filtered_data.sort_values(by=['country', 'year'], inplace=True)

        # Create 'RowNumber' within each 'Category' partition
        filtered_data['RowNumber'] = filtered_data.groupby('country').cumcount() + 1

        # Find the maximum RowNumber for each Country
        max_rownumber = filtered_data.groupby('country')['RowNumber'].max().reset_index()

        # Merge to get rows with the max RowNumber per country
        filtered_data = pd.merge(filtered_data, max_rownumber, on=['country', 'RowNumber'], how='inner')

        return filtered_data

    def GDP_clean(self):
        # Copy the data to avoid changing the original DataFrame outside the function scope
        new_GDP = self.GDP.copy()

        #rename columns from the new_GDP dataset
        new_GDP.rename(columns={
                        'Country/Area': 'country',
                        'Imports of goods and services': 'total_imports',
                        'Gross Domestic Product (GDP)' : 'GDP_USD',
                        'Year':'year_gdp'
                    },inplace=True)
        return new_GDP


## Merge extracted data for working population, avg weekly working hours and GDP

In [7]:
"""
This section is to merge both the working population  and average weekly hour dataframe
Calculate the average yearly hour by multiplying the weekly hours by 52
and also calculating the GDP per hour
"""

# Get Dataframe from GDPPerHour class object
gdp_per_hour_class = GDPPerHour(wp_df,wh_df,GDP)
cleaned_wh = gdp_per_hour_class.Clean_wh_data()
cleaned_wp = gdp_per_hour_class.Clean_wp_data()
new_GDP = gdp_per_hour_class.GDP_clean()

#Merge the working population data with the hours worked data
filtered_cleaned_wp = cleaned_wp.loc[:,['country','year','no_working_population']]
filtered_cleaned_wh = cleaned_wh.loc[:,['country','year','average_yearly_hours','average_weekly_hours']]
merged_data=pd.merge(filtered_cleaned_wp,filtered_cleaned_wh, on='country',how='left')
merged_data.sort_values(by='country')

#Calculate the total annual labour hours and total weekly labour hours
merged_data['total_yearly_hours'] = merged_data['no_working_population'] * merged_data['average_yearly_hours']
merged_data['total_weekly_hours'] = merged_data['no_working_population'] * merged_data['average_weekly_hours']

#Replace country names to match worldbank country name in the 'country' column
marged_data_replacements = {
               'United States of America': 'United States',
                'Eswatini': 'Kingdom of Eswatini',
                'Iran (Islamic Republic of)': 'Iran, Islamic Republic of',
                'North Macedonia': 'Republic of North Macedonia',
               'Occupied Palestinian Territory': 'Palestine',
                'United States Virgin Islands':'Br. Virgin Isds',
    }
merged_data['country'] = merged_data['country'].replace(marged_data_replacements, regex=True)


new_GDP_replacements = {
               'State of Palestine': 'Palestine',
               'United Republic of Tanzania: Mainland': 'Tanzania, United Republic of ',
    }

new_GDP['country'] = new_GDP['country'].replace(new_GDP_replacements,regex=True)


#rename year column from the merged_data dataset
merged_data.rename(columns={
            'year_x': 'year_total_working_hours'
                },inplace=True)

#This is to merge the marged data with the GDP data to get the final_datasets which will be worked on to get the GDP/hours
filtered_merged_date = merged_data.loc[:,['country','year_total_working_hours','total_yearly_hours']]
filtered_new_GDP = new_GDP.loc[:,['country','year_gdp','GDP_USD']]
final_merged_data=pd.merge(filtered_merged_date,filtered_new_GDP, on=['country'],how='left')

#Filter out countries that do not have data for total hours worked
final_merged_data = final_merged_data[final_merged_data['total_yearly_hours'].notna()]

#set countries that do not have gdp to zero for now
final_merged_data['GDP_USD'] = final_merged_data['GDP_USD'].fillna(0)

#Calculate GDP per hour
final_merged_data['GDP_per_hour'] = final_merged_data['GDP_USD'] / final_merged_data['total_yearly_hours']

#Merge country database to get the contry_ISO3 code
final_merged_data=pd.merge(final_merged_data,country_code_db, on=['country'],how='left')
final_merged_data

C:\Users\PC\AppData\Local\Temp\ipykernel_12956\4015528534.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.sort_values(by=['country', 'year'], inplace=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_12956\4015528534.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['RowNumber'] = filtered_data.groupby('country').cumcount() + 1
C:\Users\PC\AppData\Local\Temp\ipykernel_12956\4015528534.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

,country,year_total_working_hours,total_yearly_hours,year_gdp,GDP_USD,GDP_per_hour,country_iso3
0,Afghanistan,2021,3.563754e+10,2022.0,1.604898e+10,0.450339,AFG
1,Albania,2020,5.337159e+09,2022.0,1.431983e+10,2.683043,ALB
2,Angola,2021,4.204829e+10,2022.0,1.089901e+11,2.592022,AGO
3,Argentina,2022,4.312327e+10,2022.0,6.487673e+11,15.044482,ARG
4,Armenia,2021,4.559262e+09,2022.0,1.423863e+10,3.123011,ARM
...,...,...,...,...,...,...,...
149,Viet Nam,2022,1.799748e+11,2022.0,3.589184e+11,1.994271,VNM
150,Wallis and Futuna,2019,1.626898e+07,NaN,0.000000e+00,0.000000,WLF
151,Yemen,2014,2.877944e+10,2022.0,1.929419e+10,0.670416,YEM
152,Zambia,2022,2.584259e+10,2022.0,2.572686e+10,0.995522,ZMB


# Products data
The United Nations Comtrade Database is one of the world’s largest and most comprehensive sources of international trade data, maintained by the United Nations Statistics Division (UNSD). It provides detailed information on trade flows between countries, covering thousands of products and services traded worldwide. - https://comtradeplus.un.org/

In [8]:
subscription_key ='f6f38dac65574cab8a6b972e0c653a8a' # trial key, for bulk key, you will need to shop for premium
pd.set_option('display.max_columns', None)
hs_data = json.load(f)
country_codes = pd.read_csv(path2)
codes = country_codes['country_code'].to_list()
print(codes)

[4, 8, 12, 16, 20, 24, 28, 31, 32, 36, 40, 44, 48, 50, 51, 52, 56, 58, 60, 64, 68, 70, 72, 76, 84, 86, 90, 92, 96, 100, 104, 108, 112, 116, 120, 124, 132, 136, 140, 144, 148, 152, 156, 162, 166, 170, 174, 175, 178, 180, 184, 188, 191, 192, 196, 200, 203, 204, 208, 212, 214, 218, 222, 226, 231, 232, 233, 238, 242, 246, 251, 258, 260, 262, 266, 268, 270, 275, 276, 278, 280, 288, 292, 296, 300, 304, 308, 316, 320, 324, 328, 332, 340, 344, 348, 352, 360, 364, 368, 372, 376, 380, 384, 388, 392, 398, 400, 404, 408, 410, 414, 417, 418, 422, 426, 428, 430, 434, 440, 442, 446, 450, 454, 458, 462, 466, 470, 478, 480, 484, 490, 496, 498, 499, 500, 504, 508, 512, 516, 520, 524, 528, 530, 531, 533, 534, 535, 540, 548, 554, 558, 562, 566, 570, 574, 579, 580, 583, 584, 585, 586, 591, 598, 600, 604, 608, 612, 616, 620, 624, 626, 634, 642, 643, 646, 652, 654, 659, 660, 662, 666, 670, 674, 678, 682, 686, 688, 690, 694, 697, 699, 702, 703, 704, 705, 706, 710, 711, 716, 724, 728, 729, 736, 740, 748, 752, 

In [9]:
def split_numbers(input_string, chunk_size):
    # Split the string by commas to get a list of numbers
    numbers = input_string.split(',')

    # Group the numbers into chunks of specified size
    chunks = [numbers[i:i + chunk_size] for i in range(0, len(numbers), chunk_size)]

    return chunks

This function extracts data from comtrade in a specified criteria. learn more from here: https://github.com/uncomtrade/comtradeapicall

In [10]:
def get_data_from_uncomtrade(reporter,partner,flowCode,frequency,data_date,hs_code,hs_2_digit=None,hs_4_digit=None,hs_6_digit=None,include_none=False):

    """
    Fetch trade data from UN Comtrade based on specified criteria.

    Parameters:
    - reporter (str or list): Reporting country or countries.
    - partner (str or list): Partner country or countries.
    - frequency (str): Data frequency (e.g., 'annual', 'monthly').
    - data_date (str): Data period(s) in a comma-separated string.
    - hs_code (list or str): HS codes to retrieve data for, or "00" for all 2-digit codes.
    - hs_2_digit, hs_4_digit, hs_6_digit (bool): Flags to include specific HS code levels.
    - include_none (bool): If False, excludes rows with None values in the output.

    Returns:
    - final_dataframe (DataFrame): Raw trade data from the API.
    - test_dataframe (DataFrame): Processed trade data with comparisons.
    """

    if isinstance(data_date, str):
        years_list = data_date.split(',')
        # Check the length of the list and truncate if necessary
        if len(years_list) > 12:
            print("Maximum 12 date periods can be used in date string per call")
            return None,None

    if isinstance(reporter, list):
        string_list = [str(item) for item in reporter]
        reporter = ','.join(map(str, reporter))
    elif isinstance(reporter, str):
        pass
    else:
        print("The reporter codes should be a list or comma seperated string.")
        return None

    if isinstance(partner, list):
        string_list = [str(item) for item in partner]
        partner = ','.join(map(str, partner))
    elif isinstance(partner, str):
        pass
    else:
        print("The partner codes should be a list or comma seperated string.")
        return None

    if isinstance(hs_code, list) or hs_code=="00":
        pass
    else:
        print(f"""HS code must be passed as a list of strings or should be "00" """)
        return None
    if not hs_code:
        result = True
    else:
        # Get the length of the first element
        first_length = len(hs_code[0])

        # Check if all elements have the same length
        result = all(len(item) == first_length for item in hs_code)

    if result==False:
        print("The hs code string should include only 2,4 or 6 digitis as list do not send mixed digits hs code.")
        return None
    ############################################################
    if hs_code=="00":
        hs_list_2_digit=[]
        for item in hs_data["results"]:
            if item["aggrLevel"]==2:
                hs_list_2_digit.append(item['id'])
        codes_string = ','.join(map(str, hs_list_2_digit))
        final_dataframe = comtradeapicall.getFinalData(subscription_key, typeCode='C', freqCode=frequency, clCode='HS', period=data_date,
                                        reporterCode=reporter, cmdCode=codes_string, flowCode=flowCode, partnerCode=partner,
                                        partner2Code=None, customsCode=None, motCode=None, maxRecords=250000,
                                        format_output='JSON', aggregateBy=None, breakdownMode='classic',
                                        countOnly=None, includeDesc=True)
    ############################################################
    else:
        string_list = [str(item) for item in hs_code]
        if len(string_list[0])==2:

            codes_string_2_digit = ','.join(map(str, hs_code))
            hs_list_4_digit=[]
            for item in hs_data["results"]:
                if item["aggrLevel"]==4 and item["parent"] in hs_code:
                    hs_list_4_digit.append(item['id'])
            codes_string_4_digit = ','.join(map(str, hs_list_4_digit))
            hs_list_6_digit=[]
            for item in hs_data["results"]:
                if item["aggrLevel"]==6 and item["parent"] in hs_list_4_digit:
                    hs_list_6_digit.append(item['id'])
            codes_string_6_digit = ','.join(map(str, hs_list_6_digit))


        elif len(string_list[0])==4:
            for item in hs_code:
                unique_first_two_digits = set()
                # Extract the first two digits
                first_two_digits = item[:2]
                # Add the first two digits to the set
                unique_first_two_digits.add(first_two_digits)
            # Convert the set to a list if needed
            unique_first_two_digits_list = list(unique_first_two_digits)
            codes_string_2_digit = ','.join(map(str, unique_first_two_digits_list))
            codes_string_4_digit=','.join(map(str, hs_code))
            hs_list_6_digit=[]
            for item in hs_data["results"]:
                if item["aggrLevel"]==6 and item["parent"] in hs_code:
                    hs_list_6_digit.append(item['id'])
            codes_string_6_digit = ','.join(map(str, hs_list_6_digit))

        elif len(string_list[0])==6:
            for item in hs_code:
                unique_first_four_digits = set()
                # Extract the first two digits
                first_four_digits = item[:4]
                # Add the first two digits to the set
                unique_first_four_digits.add(first_four_digits)
            # Convert the set to a list if needed
            unique_first_four_digits_list = list(unique_first_four_digits)
            codes_string_4_digit = ','.join(map(str, unique_first_four_digits_list))
            for item in unique_first_four_digits_list:
                unique_first_two_digits = set()
                # Extract the first two digits
                first_two_digits = item[:2]
                # Add the first two digits to the set
                unique_first_two_digits.add(first_two_digits)
            # Convert the set to a list if needed
            unique_first_two_digits_list = list(unique_first_two_digits)
            codes_string_2_digit = ','.join(map(str, unique_first_two_digits_list))
            codes_string_6_digit=','.join(map(str, hs_code))

        else:
            print("The hs code string should include only 2,4 or 6 digitis as list do not send mixed digits hs code.")
            return None
        if(hs_2_digit==False and hs_4_digit==False and hs_6_digit==False):
            print("At least one of the digit selections must be True")
            return None
        codes_string=""
        if(hs_2_digit==True):
            codes_string+=codes_string_2_digit+","
        if(hs_4_digit==True):
            codes_string+=codes_string_4_digit+","
        if(hs_6_digit==True):
            codes_string+=codes_string_6_digit
        codes_string = codes_string.rstrip(',')
        print(f"""Final codes_string= {codes_string} """)
        chunk_size = 100
        chunks = split_numbers(codes_string, chunk_size)
        final_dataframe=pd.DataFrame()
        for chunk in range(0,len(chunks),1):
            string_numbers = list(map(str, chunks[chunk]))
            codes_string = ','.join(map(str, string_numbers))
            mydf = comtradeapicall.getFinalData(subscription_key, typeCode='C', freqCode=frequency, clCode='HS', period=data_date,
                                        reporterCode=reporter, cmdCode=codes_string, flowCode=flowCode, partnerCode=partner,
                                        partner2Code=None, customsCode=None, motCode=None, maxRecords=250000,
                                        format_output='JSON', aggregateBy=None, breakdownMode='classic',
                                        countOnly=None, includeDesc=True)
            final_dataframe=pd.concat([final_dataframe,mydf])
        final_dataframe=final_dataframe.reset_index(drop=True)
        return final_dataframe
    ##################################################################### Added for non equal price  check
    columns = ['cifvalue', 'fobvalue', 'primaryValue']
    # Initialize the result to True

    # Initialize an empty DataFrame to store rows with differences
    rows_with_differences = []

    # Loop through each row in the final_dataframe
    for _, row in final_dataframe.iterrows():
        # Extract the values for the specified columns
        values = [row[col] for col in columns]

        # Filter out NaN values and zeros
        non_nan_values = [value for value in values if pd.notna(value) and value != 0.0]

        # Check if all non-NaN values are the same
        if len(set(non_nan_values)) > 1:
            # If there are differences, add the row to the list
            rows_with_differences.append(row)

    # Concatenate all rows with differences into a DataFrame
    failures_dataframe = pd.DataFrame(rows_with_differences)

    # Check if any differences were found and output the result
    if not failures_dataframe.empty:
        print("There are differences in the price columns")

    ####################################################################
    test_dataframe=pd.DataFrame()
    periods_list = final_dataframe['period'].unique().tolist()
    for periods in periods_list:
        filtered_df=final_dataframe[final_dataframe["period"]==periods]

        reporter_numbers = filtered_df['reporterCode'].unique().tolist()
        for reporter_loop in reporter_numbers:
            unique_numbers = filtered_df[filtered_df["reporterCode"]==reporter_loop]['cmdCode'].unique().tolist()
            for hs_codes in unique_numbers:
                mydf=filtered_df[(filtered_df["cmdCode"]==hs_codes)&(filtered_df["reporterCode"]==reporter_loop)]
                unique_partners = mydf['partnerCode'].unique().tolist()
                if len(unique_partners)>1:
                    if unique_partners[0] == 0:
                        unique_partners = unique_partners[1:]

                for partners in unique_partners:
                    mydf=filtered_df[(filtered_df["cmdCode"]==hs_codes)&(filtered_df["reporterCode"]==reporter_loop)&(filtered_df["partnerCode"]==partners)]
                    # Check if mydf is empty
                    if mydf.empty:
                        import_quantity = None
                        export_quantity = None
                        import_amount= None
                        export_amount= None
                        unit_type = None
                        difference = None
                        balance_status = None
                        Hs_description=None
                        country_tag=None
                        reporter=None
                        partner=None
                        difference_amount = None
                        balance_status_dollars = None
                        total_parent=None
                        period_data=None
                    else:
                        flow_list=mydf["flowCode"].unique().tolist()
                        if "X" not in flow_list or "M" not in flow_list:
                            continue

                        try:
                            if mydf[mydf['flowCode'] == 'M']['qty'].values[0]!=0 and mydf[mydf['flowCode'] == 'X']['qty'].values[0]!=0 :
                                import_quantity = mydf[mydf['flowCode'] == 'M']['qty'].values[0]
                                export_quantity = mydf[mydf['flowCode'] == 'X']['qty'].values[0]
                                unit_type = mydf[mydf['flowCode'] == 'X']['qtyUnitAbbr'].values[0] if not mydf[mydf['flowCode'] == 'X']['qtyUnitAbbr'].empty else None
                            else:
                                import_quantity = mydf[mydf['flowCode'] == 'M']['altQty'].values[0] if not mydf[mydf['flowCode'] == 'M']['altQty'].empty else None
                                export_quantity = mydf[mydf['flowCode'] == 'X']['altQty'].values[0] if not mydf[mydf['flowCode'] == 'X']['altQty'].empty else None
                                unit_type = mydf[mydf['flowCode'] == 'X']['altQtyUnitAbbr'].values[0] if not mydf[mydf['flowCode'] == 'X']['altQtyUnitAbbr'].empty else None
                            ################################################ Under this part new value selector selects any not NaN
                            columns = ['cifvalue', 'fobvalue', 'primaryValue']
                            # Initialize variables
                            import_amount = None
                            export_amount = None
                            # Check for the first non-NaN, non-0.0 value for 'M'
                            for col in columns:
                                values = mydf[mydf['flowCode'] == 'M'][col].values
                                if len(values) > 0 and pd.notna(values[0]) and values[0] != 0.0:
                                    import_amount = values[0]
                                    break
                            # Check for the first non-NaN, non-0.0 value for 'X'
                            for col in columns:
                                values = mydf[mydf['flowCode'] == 'X'][col].values
                                if len(values) > 0 and pd.notna(values[0]) and values[0] != 0.0:
                                    export_amount = values[0]
                                    break
                            # If either import_amount or export_amount is None, set both to None
                            if import_amount is None or export_amount is None:
                                import_amount = None
                                export_amount = None

                        except IndexError as e:
                            print("IndexError occurred:", e)
                            print("DataFrame mydf:")
                            return final_dataframe,mydf


                        Hs_description=mydf[mydf['flowCode'] == 'X']['cmdDesc'].values[0] if not mydf[mydf['flowCode'] == 'X']['cmdDesc'].empty else None
                        country_tag = mydf[mydf['flowCode'] == 'X']['reporterISO'].values[0]
                        reporter = mydf[mydf['flowCode'] == 'X']['reporterCode'].values[0]
                        partner_tag= mydf[mydf['flowCode'] == 'X']['partnerDesc'].values[0]
                        partner=mydf[mydf['flowCode'] == 'X']['partnerCode'].values[0]
                        # Calculate the differences and balance status
                        if export_quantity is not None and import_quantity is not None:
                            difference = export_quantity - import_quantity
                            balance_status = "Positive" if difference > 0 else "Negative"
                        else:
                            difference = None
                            balance_status = None

                        if export_amount is not None and import_amount is not None:
                            difference_amount = export_amount - import_amount
                            balance_status_dollars = "Positive" if difference_amount > 0 else "Negative"
                        else:
                            difference_amount = None
                            balance_status_dollars = None


                    if include_none==False:
                        if difference==None and difference_amount==None:
                            continue

                    total_parent=None
                    if len(hs_codes)>2:
                        total_parent=hs_codes[:2]
                    data = pd.DataFrame([{
                    "Period": periods,
                    "HS Code": hs_codes,
                    "Parent": total_parent,
                    "Reporter Code": reporter,
                    "Reporter": country_tag,
                    "Partner Code": partner,
                    "Partner": partner_tag,
                    "HS_Description": Hs_description,
                    "Unit Type": unit_type,
                    "Import Quantitiy": import_quantity,
                    "Export Quantitiy": export_quantity,
                    "Import/Export Diff Quantity": difference,
                    "Import/Export Balance Quantity": balance_status,
                    "Import Amount Dollars": import_amount,
                    "Export Amount Dollars": export_amount,
                    "Import/Export Diff Dollars": difference_amount,
                    "Import/Export Balance Dollars": balance_status_dollars
                }])
                    test_dataframe = pd.concat([test_dataframe, data], ignore_index=True)
    if len(failures_dataframe)>0:
        return final_dataframe,test_dataframe
    else:
        return final_dataframe,test_dataframe

Get all the HS codes data reportered

In [11]:
uncomtrade_data_hs_2,balanced_dataframe_hs_2=get_data_from_uncomtrade(reporter="484",partner=codes, flowCode ="X",frequency="A",data_date="2022",hs_code="00",hs_2_digit=True,hs_4_digit=False,hs_6_digit=False)

In [12]:
positive_top_5_items=uncomtrade_data_hs_2["cmdCode"].unique().tolist()

> This function retrieves all export or import data for a specified country. The primary arguments to specify include the reporter code, partner code, year, and flow code (indicating export or import).

> To select the level of product detail, set the relevant argument for the HS code level:

> [2-Digit HS Codes: Set hs_2_digit=True
4-Digit HS Codes: Set hs_4_digit=True
6-Digit HS Codes: Set hs_6_digit=True]

> Now, let’s retrieve data on U.S. imports for the year 2022.


In [13]:
usa_imports_2022=get_data_from_uncomtrade(reporter="842",partner=codes,data_date="2022",flowCode='M', frequency="A", hs_code=positive_top_5_items,hs_2_digit=True,hs_4_digit=False,hs_6_digit=False)

Final codes_string= 01,02,03,04,05,06,07,08,09,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,99 


Exploring the trade data  by diplaying 5 random rows

In [14]:
usa_imports_2022.sample(n=2) # show 2 random rows

,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,flowDesc,partnerCode,partnerISO,partnerDesc,partner2Code,partner2ISO,partner2Desc,classificationCode,classificationSearchCode,isOriginalClassification,cmdCode,cmdDesc,aggrLevel,isLeaf,customsCode,customsDesc,mosCode,motCode,motDesc,qtyUnitCode,qtyUnitAbbr,qty,isQtyEstimated,altQtyUnitCode,altQtyUnitAbbr,altQty,isAltQtyEstimated,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate
6761,C,A,20220101,2022,52,2022,842,USA,USA,M,Import,716,ZWE,Zimbabwe,0,W00,World,H6,HS,True,64,Footwear; gaiters and the like; parts of such ...,2,False,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,False,-1,N/A,0.0,False,0.0,True,0.0,False,155359.0,143815.0,155359.0,4,False,True
9941,C,A,20220101,2022,52,2022,842,USA,USA,M,Import,144,LKA,Sri Lanka,0,W00,World,H6,HS,True,93,Arms and ammunition; parts and accessories the...,2,False,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,False,-1,N/A,0.0,False,0.0,False,0.0,False,77009.0,72857.0,77009.0,0,False,True


In [15]:
# filter out the most important variables in the dataframe
usa_imports_2022= usa_imports_2022[['refYear','reporterISO','reporterDesc','partnerDesc','partnerISO','cmdCode','cmdDesc', 'fobvalue']]

In [16]:
usa_imports_2022.sample(n=5)

,refYear,reporterISO,reporterDesc,partnerDesc,partnerISO,cmdCode,cmdDesc,fobvalue
6803,2022,USA,USA,Sri Lanka,LKA,65,Headgear and parts thereof,11201340.0
9997,2022,USA,USA,Spain,ESP,93,Arms and ammunition; parts and accessories the...,118458441.0
9762,2022,USA,USA,Congo,COG,91,Clocks and watches and parts thereof,42215.0
5342,2022,USA,USA,Mozambique,MOZ,48,"Paper and paperboard; articles of paper pulp, ...",2700.0
2272,2022,USA,USA,Gambia,GMB,19,"Preparations of cereals, flour, starch or milk...",6663.0


# what is the total import value of all goods imported to the USA?

In [17]:
# calculate the total import value
total_imports_2022 = usa_imports_2022['fobvalue'].sum()
print(f"Total import value for the year 2022: {total_imports_2022}")

Total import value for the year 2022: 3243131340182.0


# What is the total export value of each country to the USA?

In [18]:
# Group by 'partnerDesc' and sum import values
partner_export_2022 = usa_imports_2022.groupby(['partnerDesc','partnerISO'])['fobvalue'].sum().reset_index()
partner_export_2022.head()

,partnerDesc,partnerISO,fobvalue
0,Afghanistan,AFG,2.254660e+07
1,Albania,ALB,1.449910e+08
2,Algeria,DZA,3.060778e+09
3,Andorra,AND,9.778800e+06
4,Angola,AGO,1.611020e+09


## what products does USA import and from which country

In [19]:
partner_cmdcode_export = usa_imports_2022.groupby(['partnerDesc','cmdCode','cmdDesc','partnerISO'])['fobvalue'].sum().reset_index()
partner_cmdcode_export

,partnerDesc,cmdCode,cmdDesc,partnerISO,fobvalue
0,Afghanistan,04,Dairy produce; birds' eggs; natural honey; edi...,AFG,14916.0
1,Afghanistan,07,Vegetables and certain roots and tubers; edible,AFG,39966.0
2,Afghanistan,08,"Fruit and nuts, edible; peel of citrus fruit o...",AFG,6111518.0
3,Afghanistan,09,"Coffee, tea, mate and spices",AFG,1506451.0
4,Afghanistan,12,Oil seeds and oleaginous fruits; miscellaneous...,AFG,105071.0
...,...,...,...,...,...
10871,Zimbabwe,94,"Furniture; bedding, mattresses, mattress suppo...",ZWE,2991.0
10872,Zimbabwe,95,"Toys, games and sports requisites; parts and a...",ZWE,1240843.0
10873,Zimbabwe,96,Miscellaneous manufactured articles,ZWE,26570.0
10874,Zimbabwe,97,Works of art; collectors' pieces and antiques,ZWE,1109077.0


### Lets see what products Japan exports to USA

In [20]:
japan_exports = partner_cmdcode_export[partner_cmdcode_export['partnerDesc'] == 'Japan']
japan_exports.head()

,partnerDesc,cmdCode,cmdDesc,partnerISO,fobvalue
5079,Japan,01,Animals; live,JPN,6609107.0
5080,Japan,02,Meat and edible meat offal,JPN,70903355.0
5081,Japan,03,"Fish and crustaceans, molluscs and other aquat...",JPN,435645055.0
5082,Japan,04,Dairy produce; birds' eggs; natural honey; edi...,JPN,1429466.0
5083,Japan,05,Animal originated products; not elsewhere spec...,JPN,1747253.0


# How many countries export to the USA?

In [21]:
# prints the number of countries exporting to usa
partner_export_2022.shape[0]

220

In [22]:
# Process the export index dataframe to match our comtrade dataframe
index_df.rename(columns={'Country Name': 'partnerDesc', 'Country Code': 'partnerISO', '2022 [YR2022]': 'index'}, inplace=True)
index_df= index_df[['partnerDesc', 'partnerISO', 'index']]

In [23]:
#Marge Comtrade data with export index
constant_df = pd.merge(partner_export_2022, index_df, on=['partnerISO'], how='left')

In [24]:
# adjust and fill in the Null values
constant_df['index'] = constant_df['index'].replace('..', '100')

# fill null value with 100
constant_df['index'] = constant_df['index'].fillna(100)

Adjustment from Current(USD) to Constant(USD). Learn more here: https://datahelpdesk.worldbank.org/knowledgebase/articles/114942-what-is-the-difference-between-current-and-constan

In [25]:
# Calculate the constant USD value at 2015
constant_df['index'] = pd.to_numeric(constant_df['index'], errors='coerce')
constant_df['constant_usd'] = constant_df['fobvalue'] / (constant_df['index']) * 100
constant_df.rename(columns={'partnerISO':'country_iso3'
                },inplace=True)

constant_df

,partnerDesc_x,country_iso3,fobvalue,partnerDesc_y,index,constant_usd
0,Afghanistan,AFG,2.254660e+07,Afghanistan,143.2,1.574483e+07
1,Albania,ALB,1.449910e+08,Albania,131.2,1.105115e+08
2,Algeria,DZA,3.060778e+09,Algeria,203.1,1.507030e+09
3,Andorra,AND,9.778800e+06,Andorra,116.4,8.401031e+06
4,Angola,AGO,1.611020e+09,Angola,220.4,7.309527e+08
...,...,...,...,...,...,...
215,Viet Nam,VNM,1.275212e+11,Viet Nam,114.3,1.115671e+11
216,Wallis and Futuna Isds,WLF,1.079010e+05,NaN,100.0,1.079010e+05
217,Yemen,YEM,3.141011e+07,"Yemen, Rep.",147.5,2.129499e+07
218,Zambia,ZMB,1.352544e+08,Zambia,163.1,8.292731e+07


# What is the Constant import value?

In [26]:
# calculate the total import value
total_imports_2022 = constant_df['constant_usd'].sum()
print(f"Total adjusted import value for the year 2022: {total_imports_2022}")

Total adjusted import value for the year 2022: 2613093856064.923


# Services data
Trade in services records the value of services exchanged between residents and non-residents of an economy, including services provided through foreign affiliates established abroad. - https://www.oecd.org/en/data/indicators/trade-in-services.html

In [27]:
# slice dataframe
usa_services = services_df[services_df['REPORTER CODE'] == 'US']
sorted_usa_services = usa_services[usa_services['FLOW'] == 'Imports'].sort_values('FLOW')

# sort by specif year
services_df_2022 = sorted_usa_services[sorted_usa_services.YEAR == 2022]
services_df_2022.head()

,FLOW,INDICATOR_CODE,REPORTER CODE,PARTNER CODE,REPORTER_COUNTRY_CODE,PARTNER_COUNTRY_CODE,REPORTING ECONOMY,PARTNER ECONOMY,SECTOR,YEAR,VALUE,UNIT
42219,Imports,SOX1,US,WL,RC840,SA000,United States of America,World,Other commercial services,2022,406552.0,Million US dollar
34676,Imports,S,US,WL,RC840,SA000,United States of America,World,Memo item: Total services,2022,713886.0,Million US dollar
49747,Imports,SC,US,WL,RC840,SA000,United States of America,World,Transport,2022,157711.0,Million US dollar
26964,Imports,SPX1,US,WL,RC840,SA000,United States of America,World,Memo item: Other services,2022,432099.0,Million US dollar
7287,Imports,SOX,US,WL,RC840,SA000,United States of America,World,Commercial services,2022,688339.0,Million US dollar


Merge the Comtrade data at constant 2015 to the GDP dataframe

In [28]:
# specify final merged data columns to be merged
new_final_merged_data = final_merged_data.loc[:,['country_iso3','GDP_per_hour']]
# Merge Comtrade Data with the GDP per hour date
marged_comtrade_gdp_per_hour = pd.merge(constant_df,new_final_merged_data, on='country_iso3', how='left')
# marged_comtrade_gdp_per_hour

Include the gdp per capita to the dataframe in order to prepare the dataset for trainning. THis trainning it necessary in order to determin the GDP per hour for countries who's data are not readily available.

In [29]:
# Function to clean ther GDP_per_capita data frame
def GDP_per_capita_clean(GDP_per_capita):
        # Copy the data to avoid changing the original DataFrame outside the function scope
        new_GDP_per_capita = GDP_per_capita.copy()

        #rename columns from the new_GDP dataset
        new_GDP_per_capita.rename(columns={
                        'Country Code': 'country_iso3',
                        '2022 [YR2022]': 'GDP_per_capita_2022'
                    },inplace=True)
        # specify final merged data columns to be merged
        GDP_per_capita_ = new_GDP_per_capita.loc[:,['country_iso3','GDP_per_capita_2022']]

        return GDP_per_capita_
new_gpd_per_capita = GDP_per_capita_clean(GDP_per_capita)

# Merge the GDP per capita with the marged_comtrade_gdp_per_hour data frame and get the data ready for trainning
marged_comtrade_gdp_per_hour_per_gdpper_capita = pd.merge(marged_comtrade_gdp_per_hour,new_gpd_per_capita, on='country_iso3', how='left')
marged_comtrade_gdp_per_hour_per_gdpper_capita

,partnerDesc_x,country_iso3,fobvalue,partnerDesc_y,index,constant_usd,GDP_per_hour,GDP_per_capita_2022
0,Afghanistan,AFG,2.254660e+07,Afghanistan,143.2,1.574483e+07,0.450339,372.615895
1,Albania,ALB,1.449910e+08,Albania,131.2,1.105115e+08,2.683043,5155.29086
2,Algeria,DZA,3.060778e+09,Algeria,203.1,1.507030e+09,NaN,4602.575608
3,Andorra,AND,9.778800e+06,Andorra,116.4,8.401031e+06,NaN,39720.951859
4,Angola,AGO,1.611020e+09,Angola,220.4,7.309527e+08,2.592022,2385.119956
...,...,...,...,...,...,...,...,...
215,Viet Nam,VNM,1.275212e+11,Viet Nam,114.3,1.115671e+11,1.994271,3658.693163
216,Wallis and Futuna Isds,WLF,1.079010e+05,NaN,100.0,1.079010e+05,0.000000,NaN
217,Yemen,YEM,3.141011e+07,"Yemen, Rep.",147.5,2.129499e+07,0.670416,1017.873109
218,Zambia,ZMB,1.352544e+08,Zambia,163.1,8.292731e+07,0.995522,1308.101838


Update countries gdp per capita that are not readily available

In [30]:
# update countries with GDP per capita 2022 as null
updates = {
    'AIA': 35374, 'IOT': 29150,
    'CXR': 24000, 'CCK': 42365,
    'COK': 17033, 'FLK': 69000,
    'ATF': 31500, 'MSR': 16461,
    'NIU': 19646, 'NFK': 24678,
    'S19': 7500, 'PCN': 18800,
    'SHN': 10693, 'SPM': 46200,
    'TKL': 6275, 'WLF': 12460,
    'VGB': 40561, 'PRK': 695.9,
    'ERI': 739.7, 'GIB': 90000,
    'GRL': 48514, 'NCL': 33190,
    'SMR': 45269, 'SSD': 1025,
    'SYR': 744.6, 'VEN': 4569
}

def update_gdp_per_capita(df, updates):
    """
    Update the 'gdp_per_capita' for specified country codes in the DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing country data.
    - updates (dict): A dictionary where keys are 'country_code' values and
                      values are the new 'gdp_per_capita' to set.

    Returns:
    - pd.DataFrame: The DataFrame with updated 'gdp_per_capita' values.
    """
    for country_code, new_gdp in updates.items():
        df.loc[df['country_iso3'] == country_code, 'GDP_per_capita_2022'] = new_gdp
    return df

# Apply the updates
marged_comtrade_gdp_per_hour_per_gdpper_capita = update_gdp_per_capita(marged_comtrade_gdp_per_hour_per_gdpper_capita, updates)
marged_comtrade_gdp_per_hour_per_gdpper_capita


,partnerDesc_x,country_iso3,fobvalue,partnerDesc_y,index,constant_usd,GDP_per_hour,GDP_per_capita_2022
0,Afghanistan,AFG,2.254660e+07,Afghanistan,143.2,1.574483e+07,0.450339,372.615895
1,Albania,ALB,1.449910e+08,Albania,131.2,1.105115e+08,2.683043,5155.29086
2,Algeria,DZA,3.060778e+09,Algeria,203.1,1.507030e+09,NaN,4602.575608
3,Andorra,AND,9.778800e+06,Andorra,116.4,8.401031e+06,NaN,39720.951859
4,Angola,AGO,1.611020e+09,Angola,220.4,7.309527e+08,2.592022,2385.119956
...,...,...,...,...,...,...,...,...
215,Viet Nam,VNM,1.275212e+11,Viet Nam,114.3,1.115671e+11,1.994271,3658.693163
216,Wallis and Futuna Isds,WLF,1.079010e+05,NaN,100.0,1.079010e+05,0.000000,12460
217,Yemen,YEM,3.141011e+07,"Yemen, Rep.",147.5,2.129499e+07,0.670416,1017.873109
218,Zambia,ZMB,1.352544e+08,Zambia,163.1,8.292731e+07,0.995522,1308.101838


# Train data set to fill up the GDP per hour

In [31]:
#Set GDP PER Hour in s=zero to NA
marged_comtrade_gdp_per_hour_per_gdpper_capita['GDP_per_hour'] = marged_comtrade_gdp_per_hour_per_gdpper_capita['GDP_per_hour'].replace(0,np.nan)
#Sort GDP Per hour in
marged_comtrade_gdp_per_hour_per_gdpper_capita

,partnerDesc_x,country_iso3,fobvalue,partnerDesc_y,index,constant_usd,GDP_per_hour,GDP_per_capita_2022
0,Afghanistan,AFG,2.254660e+07,Afghanistan,143.2,1.574483e+07,0.450339,372.615895
1,Albania,ALB,1.449910e+08,Albania,131.2,1.105115e+08,2.683043,5155.29086
2,Algeria,DZA,3.060778e+09,Algeria,203.1,1.507030e+09,NaN,4602.575608
3,Andorra,AND,9.778800e+06,Andorra,116.4,8.401031e+06,NaN,39720.951859
4,Angola,AGO,1.611020e+09,Angola,220.4,7.309527e+08,2.592022,2385.119956
...,...,...,...,...,...,...,...,...
215,Viet Nam,VNM,1.275212e+11,Viet Nam,114.3,1.115671e+11,1.994271,3658.693163
216,Wallis and Futuna Isds,WLF,1.079010e+05,NaN,100.0,1.079010e+05,NaN,12460
217,Yemen,YEM,3.141011e+07,"Yemen, Rep.",147.5,2.129499e+07,0.670416,1017.873109
218,Zambia,ZMB,1.352544e+08,Zambia,163.1,8.292731e+07,0.995522,1308.101838


split the marged_comtrade_gdp_per_hour_per_gdpper_capita variable dataframe to exclude countries whos GDP per hour is NaN

In [32]:
#split marged_comtrade_gdp_per_hour_per_gdpper_capita to test and prediction data
test_data_gdp_per_hour = marged_comtrade_gdp_per_hour_per_gdpper_capita[marged_comtrade_gdp_per_hour_per_gdpper_capita['GDP_per_hour'].notna()]
test_data_gdp_per_hour

,partnerDesc_x,country_iso3,fobvalue,partnerDesc_y,index,constant_usd,GDP_per_hour,GDP_per_capita_2022
0,Afghanistan,AFG,2.254660e+07,Afghanistan,143.2,1.574483e+07,0.450339,372.615895
1,Albania,ALB,1.449910e+08,Albania,131.2,1.105115e+08,2.683043,5155.29086
4,Angola,AGO,1.611020e+09,Angola,220.4,7.309527e+08,2.592022,2385.119956
7,Argentina,ARG,6.937554e+09,Argentina,135.7,5.112420e+09,15.044482,12940.738435
8,Armenia,ARM,8.066873e+07,Armenia,145.5,5.544243e+07,3.123011,5116.205756
...,...,...,...,...,...,...,...,...
214,Venezuela,VEN,4.130028e+08,"Venezuela, RB",191.9,2.152177e+08,2.115282,4569
215,Viet Nam,VNM,1.275212e+11,Viet Nam,114.3,1.115671e+11,1.994271,3658.693163
217,Yemen,YEM,3.141011e+07,"Yemen, Rep.",147.5,2.129499e+07,0.670416,1017.873109
218,Zambia,ZMB,1.352544e+08,Zambia,163.1,8.292731e+07,0.995522,1308.101838


In [33]:
# Split NaN dataframe
marged_comtrade_gdp_per_hour_per_gdpper_capita_na = marged_comtrade_gdp_per_hour_per_gdpper_capita[marged_comtrade_gdp_per_hour_per_gdpper_capita['GDP_per_hour'].isna()]
marged_comtrade_gdp_per_hour_per_gdpper_capita_na

,partnerDesc_x,country_iso3,fobvalue,partnerDesc_y,index,constant_usd,GDP_per_hour,GDP_per_capita_2022
2,Algeria,DZA,3.060778e+09,Algeria,203.1,1.507030e+09,NaN,4602.575608
3,Andorra,AND,9.778800e+06,Andorra,116.4,8.401031e+06,NaN,39720.951859
5,Anguilla,AIA,5.712335e+06,NaN,100.0,5.712335e+06,NaN,35374
6,Antigua and Barbuda,ATG,8.824366e+06,Antigua and Barbuda,109.6,8.051429e+06,NaN,17335.04704
9,Aruba,ABW,1.455013e+07,Aruba,104.0,1.399051e+07,NaN,32492.178878
...,...,...,...,...,...,...,...,...
202,Turkmenistan,TKM,4.633372e+07,Turkmenistan,328.9,1.408748e+07,NaN,8489.722266
203,Turks and Caicos Isds,TCA,1.202878e+07,Turks and Caicos Islands,122.4,9.827437e+06,NaN,20323.148677
207,Ukraine,UKR,1.476971e+09,Ukraine,209.4,7.053349e+08,NaN,2042.639038
212,Uzbekistan,UZB,5.901554e+07,Uzbekistan,166.9,3.535982e+07,NaN,3473.361643


In [34]:
# Create new dataframe from test and na data

#Test
test_data = test_data_gdp_per_hour.loc[:,['GDP_per_capita_2022','GDP_per_hour']]
test_data

,GDP_per_capita_2022,GDP_per_hour
0,372.615895,0.450339
1,5155.29086,2.683043
4,2385.119956,2.592022
7,12940.738435,15.044482
8,5116.205756,3.123011
...,...,...
214,4569,2.115282
215,3658.693163,1.994271
217,1017.873109,0.670416
218,1308.101838,0.995522


In [35]:
#prediction dataframe
predict_data = marged_comtrade_gdp_per_hour_per_gdpper_capita_na.loc[:,['GDP_per_capita_2022','GDP_per_hour']]
predict_data

,GDP_per_capita_2022,GDP_per_hour
2,4602.575608,NaN
3,39720.951859,NaN
5,35374,NaN
6,17335.04704,NaN
9,32492.178878,NaN
...,...,...
202,8489.722266,NaN
203,20323.148677,NaN
207,2042.639038,NaN
212,3473.361643,NaN


check for correlation between the GDP per hour and the GDP per capita

In [36]:
# Determin Correlation
correlation = test_data.corr()
correlation

,GDP_per_capita_2022,GDP_per_hour
GDP_per_capita_2022,1.000000,0.992003
GDP_per_hour,0.992003,1.000000


Train test dataframe using the Random forest learner regression model from Skitlean

In [37]:
# Setup random seed
np.random.seed(42)

# Make the data
X = test_data.drop("GDP_per_hour", axis=1)
y = test_data["GDP_per_hour"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Instantiate Random Forest Regressor
rf = RandomForestRegressor()

# Fit the model to the data (training the machine learning model)
rf.fit(X_train, y_train)

# Evaluate the Random Forest Regressor (use the patterns the model has learned)
rf.score(X_test, y_test)

0.9825574013329578

Perform a cross-validation on the test dataset to find the most optimal estimator to improve the model

In [38]:
# Model hyper paramiterization
# Use GridSearchCV with our regression Pipeline

pipe_grid = {
    "n_estimators": [100, 1000],
    "max_depth": [None, 5],
    "max_features": ["sqrt"],
    "min_samples_split": [2, 4]
}

gs_model = GridSearchCV(rf, pipe_grid, cv=5, verbose=2)
gs_model.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=1000; total time=   2.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=1000; total time=   2.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=1000; total time=   2.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=1000; total time=   1.9s
[CV] END max_depth

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 5], 'max_features': ['sqrt'],
                         'min_samples_split': [2, 4],
                         'n_estimators': [100, 1000]},
             verbose=2)

In [39]:
# Score the test data using the GridsearchCV
gs_model.score(X_test, y_test)

0.9845958690514875

Make prediction on the X_test dataframe

In [40]:
 # Make predictions using RandomForestRegressor
y_preds = gs_model.predict(X_test)
y_preds

array([ 1.52956272,  0.69877681,  2.75172118,  8.79120628,  2.64732858,
        2.32316719,  8.44331552,  0.91335944,  2.35108091,  0.59814197,
        0.97303447,  3.06472654,  1.74930306,  0.94609573,  4.18687719,
        1.55907891, 10.25460268,  0.95471092, 64.19946973,  1.11560372,
        0.97303447,  3.53978437,  2.09407468,  9.90305077,  1.59635874,
       29.28256182, 29.19052645,  2.7905227 ,  1.26748613])

In [41]:
# Compare the predictions to the truth
mean_absolute_error(y_test, y_preds)

np.float64(0.7943707220798258)

Make predictions on the predict datasets

In [42]:
per_hour_pred = predict_data.drop('GDP_per_hour',axis=1)
per_hour_pred.columns
#make prediction on prediction data set
y_preds_rf = gs_model.predict(per_hour_pred)
y_preds_rf

array([ 2.55123141, 27.40655218, 20.3884812 , 10.62347321, 21.05553164,
       41.51439333,  3.03031572, 14.93280124, 10.24671792, 68.80932486,
        3.53581468, 16.81406531, 27.63659741,  0.92170578, 57.31325724,
        0.69877681,  0.57552091,  6.96804039, 29.28256182, 21.05553164,
       14.93280124, 29.12654281,  4.95539259, 10.6546664 ,  0.54552292,
        0.81068389,  2.25350992,  4.31354945,  3.53872535,  0.59814197,
        2.52931463, 45.26124722,  3.06472654, 20.39176131, 11.68817595,
        4.23919837, 64.19946973, 32.25865116,  2.55123141,  0.85558203,
        3.03031572, 21.48424934,  6.80909167, 17.48377465,  5.66420316,
        6.85398101,  2.18718511, 29.12654281, 12.37594593, 15.13523646,
        3.85113274, 11.5416434 ,  4.96095255,  1.91804369, 11.5416434 ,
       41.60966858,  6.15613312, 12.29958874, 31.02473113,  5.90316546,
       27.3289192 ,  1.14991451, 13.38248327, 42.89083734, 21.02177393,
        1.09493579,  4.00532294,  0.64692208,  0.85064232,  3.53

In [43]:
#Create GDP per hours on predit_gdp_hour
predict_data['GDP_per_hour'] = y_preds_rf
predict_data

,GDP_per_capita_2022,GDP_per_hour
2,4602.575608,2.551231
3,39720.951859,27.406552
5,35374,20.388481
6,17335.04704,10.623473
9,32492.178878,21.055532
...,...,...
202,8489.722266,5.901985
203,20323.148677,12.257271
207,2042.639038,1.606427
212,3473.361643,2.311192


Merge the predicted gdp per hour result with the merged data set to fill the NaN in the dataset

In [44]:
# Update GDP Per Hour in the marged_comtrade_gdp_per_hour_per_gdpper_capita dataframe
marged_comtrade_gdp_per_hour_per_gdpper_capita.update(predict_data)
marged_comtrade_gdp_per_hour_per_gdpper_capita

,partnerDesc_x,country_iso3,fobvalue,partnerDesc_y,index,constant_usd,GDP_per_hour,GDP_per_capita_2022
0,Afghanistan,AFG,2.254660e+07,Afghanistan,143.2,1.574483e+07,0.450339,372.615895
1,Albania,ALB,1.449910e+08,Albania,131.2,1.105115e+08,2.683043,5155.29086
2,Algeria,DZA,3.060778e+09,Algeria,203.1,1.507030e+09,2.551231,4602.575608
3,Andorra,AND,9.778800e+06,Andorra,116.4,8.401031e+06,27.406552,39720.951859
4,Angola,AGO,1.611020e+09,Angola,220.4,7.309527e+08,2.592022,2385.119956
...,...,...,...,...,...,...,...,...
215,Viet Nam,VNM,1.275212e+11,Viet Nam,114.3,1.115671e+11,1.994271,3658.693163
216,Wallis and Futuna Isds,WLF,1.079010e+05,NaN,100.0,1.079010e+05,8.791206,12460
217,Yemen,YEM,3.141011e+07,"Yemen, Rep.",147.5,2.129499e+07,0.670416,1017.873109
218,Zambia,ZMB,1.352544e+08,Zambia,163.1,8.292731e+07,0.995522,1308.101838


In [45]:
#Update data type of the gdp_per_capita column from object to float64
marged_comtrade_gdp_per_hour_per_gdpper_capita['GDP_per_capita_2022'] = pd.to_numeric(marged_comtrade_gdp_per_hour_per_gdpper_capita['GDP_per_capita_2022'], errors='coerce')
marged_comtrade_gdp_per_hour_per_gdpper_capita.dtypes

partnerDesc_x           object
country_iso3            object
fobvalue               float64
partnerDesc_y           object
index                  float64
constant_usd           float64
GDP_per_hour           float64
GDP_per_capita_2022    float64
dtype: object

# See how well the prediction performed between the GDP per hour and the GDP per capita in order to check for outliners

In [46]:
#Build scatterplot of gdp_per_hour vs gdp_per_capita to show the performance of the regression model
fig = px.scatter(marged_comtrade_gdp_per_hour_per_gdpper_capita, x="GDP_per_capita_2022", y="GDP_per_hour", color='country_iso3', text='country_iso3')
fig.update_layout(
  title="GDP_per_hour vs. GDP_per_capita",
  xaxis_title="GDP_per_capita",
  yaxis_title="GDP_per_hour",
)
# Prepare the data for regression
x = marged_comtrade_gdp_per_hour_per_gdpper_capita['GDP_per_capita_2022']
y = marged_comtrade_gdp_per_hour_per_gdpper_capita['GDP_per_hour']

# Add a constant to the predictor variable for the intercept
x_with_constant = sm.add_constant(x)

# Fit the ordinary least squares regression model
model = sm.OLS(y, x_with_constant).fit()

# Get the regression line
x_reg = pd.Series(np.linspace(x.min(), x.max(), num=100))
x_reg_with_constant = sm.add_constant(x_reg)
y_reg = model.predict(x_reg_with_constant)

# Add trendline to the plot
fig.add_scatter(x=x_reg, y=y_reg, mode='lines', name='Trendline', line=dict(color='blue'))
fig.show()

# calculate the toal_hours_needed_to_produce_in_USA which will be the constant_usd divided by the GDP_per_hour


In [47]:
#calculate total hours for products
total_hours_produce_in_USA = marged_comtrade_gdp_per_hour_per_gdpper_capita
total_hours_produce_in_USA['toal_hours_needed_to_produce_in_USA'] = total_hours_produce_in_USA['constant_usd'] / total_hours_produce_in_USA['GDP_per_hour']
total_hours_products = total_hours_produce_in_USA['toal_hours_needed_to_produce_in_USA'].sum()

#print result of total hours for products
print(f'Total hours to produce products in the USA: {total_hours_products}.')

#Calculate for Services
def services(df):

    #Get the service 2022 data set
    services = df.copy() #make a capy of the dataframe

    #Get the service value
    service_value = services[services['SECTOR'] == 'Memo item: Total services']

    #Get average GDP_per_hour of the exporting countries
    avarage_gdp_per_hour = marged_comtrade_gdp_per_hour_per_gdpper_capita['GDP_per_hour'].mean() # This will be used to get the total hours used by dividing the service constant value by the average_gdp_per_hours
    avarage_index_for_service = marged_comtrade_gdp_per_hour_per_gdpper_capita['index'].mean() # This will be used to discount the service value to constant 2015

    #main calculation
    service_value['total_service_hours'] = (((service_value['VALUE'] * 1000000) / avarage_index_for_service) * 100)  / avarage_gdp_per_hour



    return service_value

total_service_hours =services(services_df_2022)

total_hours_services = total_service_hours['total_service_hours'].sum()

#print result of total hours for services
print(f'Total hours to produce services in the USA: {total_hours_services}.')


Total hours to produce products in the USA: 421853272043.6156.
Total hours to produce services in the USA: 47951105623.27837.


C:\Users\PC\AppData\Local\Temp\ipykernel_12956\629666990.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Answers to the objectives

In [48]:
# Deatermine total hours to work in order to produce internally
total_hours_to_be_worked = total_hours_products + total_hours_services

# Get current yearly hours worked to produce the GDP in the US at constant 2015
total_hours_worked_US = final_merged_data[final_merged_data['country_iso3'] == 'USA']['total_yearly_hours'].sum()

# Percentage hours increase
hours_increase_percentage = (total_hours_to_be_worked / total_hours_worked_US) *100

# Get working population in the USA
US_working_population = cleaned_wp[cleaned_wp['country'] == 'United States of America']['no_working_population'].sum()

#Get new population workforce to produce internally
new_workforce_to_product_internally = US_working_population * (1 + (hours_increase_percentage / 100)).sum()

# print results
print(f' Total hours to be worked to produce products internally: {total_hours_products}')
print(f' Total hours to be worked to produce services internally: {total_hours_services}')
print(f' Total hours to be worked to produce products and services internally: {total_hours_to_be_worked}')
print(f' Total hours worked to produce products and services at current GDP 2022: {total_hours_worked_US}')
print(f' Percentage increase in hours worked: {hours_increase_percentage:.2f}')
print(f' US current workforce: {US_working_population}')
print(f' New workforce to produce imports in the USA: {new_workforce_to_product_internally}')

 Total hours to be worked to produce products internally: 421853272043.6156
 Total hours to be worked to produce services internally: 47951105623.27837
 Total hours to be worked to produce products and services internally: 469804377666.894
 Total hours worked to produce products and services at current GDP 2022: 523670559867.00006
 Percentage increase in hours worked: 89.71
 US current workforce: 263973465.00000003
 New workforce to produce imports in the USA: 500793899.3516958


# Provide Answer to Objective using at product level


In [49]:
# rename column partnerISO to country_iso3
partner_cmdcode_export.rename(columns={'partnerISO': 'country_iso3'}, inplace=True)
partner_cmdcode_export.head()

,partnerDesc,cmdCode,cmdDesc,country_iso3,fobvalue
0,Afghanistan,04,Dairy produce; birds' eggs; natural honey; edi...,AFG,14916.0
1,Afghanistan,07,Vegetables and certain roots and tubers; edible,AFG,39966.0
2,Afghanistan,08,"Fruit and nuts, edible; peel of citrus fruit o...",AFG,6111518.0
3,Afghanistan,09,"Coffee, tea, mate and spices",AFG,1506451.0
4,Afghanistan,12,Oil seeds and oleaginous fruits; miscellaneous...,AFG,105071.0


In [50]:
# select column that are necessary for partnet_CmdCode_export with the marged_comtrade_gdp_per_hour_per_gdpper_capita
final_merged_gdp_per_hour_new = marged_comtrade_gdp_per_hour_per_gdpper_capita.loc[:,['country_iso3','index','GDP_per_hour']]
final_partner_cmdcode_export = partner_cmdcode_export.loc[:,['partnerDesc','country_iso3','cmdCode','cmdDesc','fobvalue']]

# Merge the partnet_CmdCode_export with the marged_comtrade_gdp_per_hour_per_gdpper_capita
marged_product_breakdown = pd.merge(final_partner_cmdcode_export,final_merged_gdp_per_hour_new, on='country_iso3', how='left')

## calculate and create constant_usd and total_hours_needed_to_produce_in_USA columns

In [57]:
# constant_usd
marged_product_breakdown['constant_usd'] = marged_product_breakdown['fobvalue'] / (marged_product_breakdown['index']) * 100

# toal_hours_needed_to_produce_in_USA
marged_product_breakdown['total_hours_needed_to_produce_in_USA'] = marged_product_breakdown['constant_usd'] / marged_product_breakdown['GDP_per_hour']
marged_product_breakdown.head()

,partnerDesc,country_iso3,cmdCode,cmdDesc,fobvalue,index,GDP_per_hour,constant_usd,toal_hours_needed_to_produce_in_USA,total_hours_needed_to_produce_in_USA
0,Afghanistan,AFG,04,Dairy produce; birds' eggs; natural honey; edi...,14916.0,143.2,0.450339,1.041620e+04,2.312968e+04,2.312968e+04
1,Afghanistan,AFG,07,Vegetables and certain roots and tubers; edible,39966.0,143.2,0.450339,2.790922e+04,6.197378e+04,6.197378e+04
2,Afghanistan,AFG,08,"Fruit and nuts, edible; peel of citrus fruit o...",6111518.0,143.2,0.450339,4.267820e+06,9.476902e+06,9.476902e+06
3,Afghanistan,AFG,09,"Coffee, tea, mate and spices",1506451.0,143.2,0.450339,1.051991e+06,2.335997e+06,2.335997e+06
4,Afghanistan,AFG,12,Oil seeds and oleaginous fruits; miscellaneous...,105071.0,143.2,0.450339,7.337360e+04,1.629297e+05,1.629297e+05


In [58]:
# list for product code
list(marged_product_breakdown['cmdCode'].unique())

['04',
 '07',
 '08',
 '09',
 '12',
 '17',
 '19',
 '20',
 '21',
 '22',
 '24',
 '25',
 '32',
 '33',
 '39',
 '40',
 '42',
 '46',
 '48',
 '49',
 '54',
 '57',
 '58',
 '61',
 '62',
 '63',
 '64',
 '68',
 '71',
 '73',
 '74',
 '76',
 '83',
 '84',
 '85',
 '90',
 '94',
 '97',
 '99',
 '03',
 '06',
 '10',
 '11',
 '15',
 '16',
 '18',
 '27',
 '30',
 '31',
 '34',
 '37',
 '38',
 '41',
 '43',
 '44',
 '55',
 '56',
 '65',
 '69',
 '70',
 '72',
 '82',
 '87',
 '92',
 '96',
 '28',
 '29',
 '81',
 '86',
 '88',
 '95',
 '01',
 '02',
 '05',
 '13',
 '14',
 '23',
 '35',
 '36',
 '45',
 '51',
 '52',
 '59',
 '60',
 '66',
 '67',
 '78',
 '80',
 '89',
 '91',
 '93',
 '26',
 '53',
 '75',
 '79',
 '50',
 '47']

In [59]:
# list for country_iso3
list(marged_product_breakdown['country_iso3'].unique())

['AFG',
 'ALB',
 'DZA',
 'AND',
 'AGO',
 'AIA',
 'ATG',
 'ARG',
 'ARM',
 'ABW',
 'AUS',
 'AUT',
 'AZE',
 'BHS',
 'BHR',
 'BGD',
 'BRB',
 'BLR',
 'BEL',
 'BLZ',
 'BEN',
 'BMU',
 'BTN',
 'BOL',
 'BIH',
 'BWA',
 'IOT',
 'VGB',
 'BRA',
 'BRN',
 'BGR',
 'BFA',
 'BDI',
 'CPV',
 'KHM',
 'CMR',
 'CAN',
 'CYM',
 'CAF',
 'TCD',
 'CHL',
 'CHN',
 'HKG',
 'MAC',
 'CXR',
 'CCK',
 'COL',
 'COM',
 'COG',
 'COK',
 'CRI',
 'HRV',
 'CUB',
 'CUW',
 'CYP',
 'CZE',
 'CIV',
 'PRK',
 'COD',
 'DNK',
 'DJI',
 'DMA',
 'DOM',
 'ECU',
 'EGY',
 'SLV',
 'GNQ',
 'ERI',
 'EST',
 'SWZ',
 'ETH',
 'FSM',
 'FLK',
 'FJI',
 'FIN',
 'ATF',
 'FRA',
 'PYF',
 'GAB',
 'GMB',
 'GEO',
 'DEU',
 'GHA',
 'GIB',
 'GRC',
 'GRL',
 'GRD',
 'GTM',
 'GIN',
 'GNB',
 'GUY',
 'HTI',
 'HND',
 'HUN',
 'ISL',
 'IND',
 'IDN',
 'IRN',
 'IRQ',
 'IRL',
 'ISR',
 'ITA',
 'JAM',
 'JPN',
 'JOR',
 'KAZ',
 'KEN',
 'KIR',
 'KWT',
 'KGZ',
 'LAO',
 'LVA',
 'LBN',
 'LSO',
 'LBR',
 'LBY',
 'LTU',
 'LUX',
 'MDG',
 'MWI',
 'MYS',
 'MDV',
 'MLI',
 'MLT',
 'MHL',


##  function to caluate labor force increase based on any product selected

In [ ]:
# new_labor_force function

def new_labor_force(df, product_code= None, country_iso3 = None):
    new_df = df.copy()

# Search by Product Code
    if product_code is not None:
        new_df = new_df[new_df['cmdCode'] == product_code]
        if new_df.empty:
          print(f'product code {product_code} does not exist')
        else:
          total_hours_worked_US = final_merged_data[final_merged_data['country_iso3'] == 'USA']['total_yearly_hours'].sum()
          product_df = new_df[new_df['cmdCode']== product_code]['total_hours_needed_to_produce_in_USA'].sum()
          cmd_desc = new_df[new_df['cmdCode'] == product_code]['cmdDesc'].iloc[0]
          hours_increase_percentage = (product_df  / total_hours_worked_US) *100
          new_labor_force = US_working_population * (1 + (hours_increase_percentage / 100))

          dic = {
              'cmdDesc': cmd_desc,
              'product_code': product_code,
              'total_hours_needed_to_produce_in_USA': product_df,
              'total_hours_worked_US': total_hours_worked_US,
              'hours_increase_percentage': hours_increase_percentage,
              'US_working_population': US_working_population,
              'new_labor_force': new_labor_force
          }

          return dic

# search by country code
    elif country_iso3 is not None:
      new_df = new_df[new_df['country_iso3'] == country_iso3]
      if new_df.empty:
        print(f'country_iso3 {country_iso3} does not exist')
      else:
        total_hours_worked_US = final_merged_data[final_merged_data['country_iso3'] == 'USA']['total_yearly_hours'].sum()
        product_df = new_df[new_df['country_iso3'] == country_iso3]['total_hours_needed_to_produce_in_USA'].sum()
        partnerDesc = new_df[new_df['country_iso3'] == country_iso3]['partnerDesc'].iloc[0]
        hours_increase_percentage = (product_df  / total_hours_worked_US) *100
        new_labor_force = US_working_population * (1 + (hours_increase_percentage / 100))

        dic = {
            'partnerDesc': partnerDesc,
            'total_hours_needed_to_produce_in_USA': product_df,
            'total_hours_worked_US': total_hours_worked_US,
            'hours_increase_percentage': hours_increase_percentage,
            'US_working_population': US_working_population,
            'new_labor_force': new_labor_force
        }

        return dic

In [61]:
product_labor_force_increase = new_labor_force(marged_product_breakdown,None,'NIU')
product_labor_force_increase

{'partnerDesc': 'Niue',
 'total_hours_needed_to_produce_in_USA': np.float64(212302.96376673187),
 'total_hours_worked_US': np.float64(523670559867.00006),
 'hours_increase_percentage': np.float64(4.0541321211689236e-05),
 'US_working_population': np.float64(263973465.00000003),
 'new_labor_force': np.float64(263973572.0183304)}